<a href="https://colab.research.google.com/github/microsoft/FLAML/blob/main/notebook/autogen_agentchat_groupchat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Generated Agent Chat: Group Chat

`flaml.autogen` offers conversable agents powered by LLM, tool or human, which can be used to perform tasks collectively via automated chat. This framwork allows tool use and human participance through multi-agent conversation.
Please find documentation about this feature [here](https://microsoft.github.io/FLAML/docs/Use-Cases/Autogen#agents).

This notebook is modified based on https://github.com/microsoft/FLAML/blob/4ea686af5c3e8ff24d9076a7a626c8b28ab5b1d7/notebook/autogen_multiagent_roleplay_chat.ipynb

## Requirements

FLAML requires `Python>=3.8`. To run this notebook example, please install flaml with the [autogen] option:
```bash
pip install flaml[autogen]
```

In [1]:
%%capture --no-stderr
# %pip install flaml[autogen]~=2.0.1

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/FLAML/docs/reference/autogen/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [2]:
from flaml import autogen

config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)
# config_list_gpt35 = autogen.config_list_from_json(
#     "OAI_CONFIG_LIST",
#     filter_dict={
#         "model": {
#             "gpt-3.5-turbo",
#             "gpt-3.5-turbo-16k",
#             "gpt-3.5-turbo-0301",
#             "chatgpt-35-turbo-0301",
#             "gpt-35-turbo-v0301",
#         },
#     },
# )

It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well). Only the gpt-4 models are kept in the list based on the filter condition.

The config list looks like the following:
```python
config_list = [
    {
        'model': 'gpt-4',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt-4',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
    {
        'model': 'gpt-4-32k',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
]
```

If you open this notebook in colab, you can upload your files by clicking the file icon on the left panel and then choose "upload file" icon.

You can set the value of config_list in other ways you prefer, e.g., loading from a YAML file.

## Construct Agents

In [3]:
llm_config = {"config_list": config_list_gpt4}
human = autogen.UserProxyAgent(
   name="Human",
   system_message="A human admin.",
   code_execution_config={"last_n_messages": 2, "work_dir": "groupchat"},
)
alice = autogen.AssistantAgent(
    name="Alice",
    llm_config=llm_config,
)
bob = autogen.AssistantAgent(
    name="Bob",
    system_message="Code and answer reviewer."
    "For code, prevent code execution if unsafe or missing important details, e.g., sort order in arxiv API. Suggest changes. Otherwise, approve and return the final code to execute."
    "For answer, carefully check the interpretation of code result and fix any errors. If the interpretation is correct, approve and return the final answer to the user.",
    llm_config=llm_config,
)
groupchat = autogen.GroupChat(agents=[human, alice, bob], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

## Start Chat

In [4]:
human.initiate_chat(manager, message="find a latest paper about gpt-4")

Human (to chat_manager):

find a latest paper about gpt-4

--------------------------------------------------------------------------------
Alice (to chat_manager):

To find the latest paper about GPT-4, we can use the arXiv API, an interface for accessing arXiv's raw metadata. We send a query specifying the subject - in this case 'GPT-4' - and retrieve the list of papers about this subject. From that list, we select the most recent paper.

Step 1: I will use Python code to send an HTTP request to the arXiv API, requesting papers about 'GPT-4'. I will print the raw results.

Step 2: I will examine the results, which are XML files containing metadata about each paper. Normally, the papers are ordered with the most recently updated paper coming first, so the latest paper about GPT-4 should be at the top of the list.

The Python code to execute this task is:

```python
import feedparser

def fetch_papers(search_query='gpt-4', max_results=10):
    base_url = 'http://export.arxiv.org/api/qu